### Análisis de Datos del Mercado de Criptomonedas con Binance API

- En la era digital actual, el mercado de criptomonedas ha emergido como un campo fascinante y dinámico, ofreciendo oportunidades únicas para el análisis de datos y la comprensión de las tendencias del mercado financiero. Nuestro proyecto se centra en explorar este sector vibrante utilizando una de las herramientas más poderosas en el mundo de las criptomonedas: la API de Binance.

- Binance, siendo una de las plataformas de intercambio de criptomonedas más grandes y reconocidas a nivel mundial, proporciona una API rica y robusta, permitiendo a los usuarios acceder a una amplia gama de datos del mercado en tiempo real. Esto incluye, pero no se limita a, datos históricos de precios, detalles de operaciones individuales, volúmenes de trading, y mucho más.

- El objetivo principal de este proyecto es aprovechar estos datos para obtener insights valiosos sobre el comportamiento del mercado de Bitcoin (BTC), la criptomoneda líder en el mercado. Nos centraremos en extraer datos históricos de precios y operaciones de BTC comparados con el USD, analizando las tendencias, volatilidades y posibles patrones de trading.

- Al final de este proyecto, esperamos tener un análisis detallado y multifacético del mercado de Bitcoin, brindando un panorama claro no solo para los entusiastas de las criptomonedas, sino también para aquellos interesados en las finanzas y la tecnología. Este desafío es una excelente oportunidad para sumergirnos en el mundo del análisis de datos financieros y abrir la puerta a futuras investigaciones y proyectos en este apasionante campo.

In [2]:
# paquetes necesarios
import requests
import pandas as pd
import hmac
import hashlib
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from decouple import config
from binance.client import Client
api_key = config('API_KEY')
api_secret = config('API_SECRET')
api_limit = 1000
requestDays = 30
limit = 1000

## Descargar precios de los ultimos 30 dias

In [3]:
# Crear una instancia del cliente de Binance
client = Client(api_key, api_secret)

# Calcular las fechas para el último mes
end_date = datetime.utcnow()
start_date = end_date - timedelta(days=requestDays)

# Convertir las fechas a milisegundos (que es el formato que la API de Binance espera)
start_date_timestamp = int(start_date.timestamp() * 1000)
end_date_timestamp = int(end_date.timestamp() * 1000)

# Obtener los datos del precio del Bitcoin para el último mes con intervalo de 1 minuto
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1MINUTE

C:\Users\German\AppData\Local\Temp\ipykernel_10096\4292302460.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_date = datetime.utcnow()


In [13]:

# Inicializar una lista para almacenar los datos
all_klines = []

# Hacer llamadas sucesivas para obtener más datos (limitando cada llamada a 500)

while start_date_timestamp < end_date_timestamp:
    klines = client.get_klines(symbol=symbol, interval=interval, startTime=start_date_timestamp, endTime=end_date_timestamp, limit=limit)
    
    # Verificar si hay datos en la respuesta
    if not klines:
        break
    
    all_klines.extend(klines)
    
    # Actualizar el startTime para la siguiente llamada
    start_date_timestamp = int((pd.to_datetime(klines[-1][0], unit='ms') + timedelta(minutes=1)).timestamp() * 1000)

# Comprobar si se recuperaron datos antes de crear el DataFrame
if all_klines:
    # Crear un DataFrame de Pandas con todos los datos
    columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    df = pd.DataFrame(all_klines, columns=columns)

    # Convertir el timestamp a formato de fecha y hora legible
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Mostrar el DataFrame
    df.info()
else:
    print("No se encontraron datos para el período especificado.")

KeyboardInterrupt: 

## Descargar trades de los ultimos 30 dias

In [12]:

# Inicializar variables
symbol = "BTCUSDT"
interval = "1m"
start_date_timestamp = 1646092800000  # 2022-03-08 00:00:00
end_date_timestamp = 1646179200000  # 2022-03-08 23:59:59
limit = 500

# Obtener trades
all_trades = []

while start_date_timestamp < end_date_timestamp:
    trades = client.get_historical_trades(symbol=symbol, startTime=start_date_timestamp, endTime=end_date_timestamp, limit=limit)

    if not trades:
        break

    all_trades.extend(trades)

    start_date_timestamp = int((pd.to_datetime(trades[-1]['timestamp'], unit='ms') + timedelta(minutes=1)).timestamp() * 1000)

# Crear DataFrame
if all_trades:
    df = pd.DataFrame(all_trades, columns=columns)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.info()
else:
    print("No se encontraron datos para el período especificado.")

BinanceAPIException: APIError(code=-1104): Not all sent parameters were read; read '1' parameter(s) but was sent '3'.